# Week 5 Problem Set

## Homework

In [1]:
%load_ext nb_mypy
%nb_mypy On

ModuleNotFoundError: No module named 'nb_mypy'

In [2]:
from typing import TypeAlias
from typing import Optional, Any, Iterator
from __future__ import annotations

Number: TypeAlias = int | float
NumberList: TypeAlias = list[int|float] 

**HW1.** Modify the class `TurtleWorld` to include the following attribute and methods:
- `movement_queue` which is an attribute of the type `Queue` to store the movement list. **Each entry of this queue is an object which has the name of the turtle (e.g. "t1") and the movement list (e.g. "ulrr"). The choice of this object is left to you, e.g. it could be a tuple (`("t1", "ulrr")`) or a dictionary with the same information or your own custom class.
- `add_movement(turtle, movement)` which adds turtle movement to the queue `movement_queue` to be run later. The argument `turtle` is a string containing the turtle's name. The argument `movement` is another string for the movement. For example, value for `turtle` can be something like `'t1'` while the value for the `movement` can be something like `'uullrrdd'`.
- `run()` which executes all the movements in the queue.

In [3]:
import math

class Coordinate:
    
    def __init__(self, x:Number=0, y:Number=0) -> None:
        self.x = x
        self.y = y
        
    @property
    def distance(self) -> float:
        return math.sqrt(self.x * self.x + self.y * self.y)
    
    def __str__(self) -> str:
        return f"({self.x}, {self.y})"

In [7]:
class Stack:
    def __init__(self) -> None:
        self.__items: list[Any] = []
        
    def push(self, item: Any):
        self.__items.append(item)

    def pop(self) -> Any:
        if (len(self.__items) == 0):
            return None
        return self.__items.pop()

    def peek(self) -> Any:
        return self.__items[-1]

    @property
    def is_empty(self) -> bool:
        return (len(self.__items) == 0)

    @property
    def size(self):
        return len(self.__items)

class Queue:
    def __init__(self) -> None:
        self.left_stack: Stack = Stack()
        self.right_stack: Stack = Stack()
        
    def enqueue(self, item: Any):
        self.left_stack.push(item)

    def dequeue(self) -> Any:
        if (self.right_stack.is_empty):
            while(self.left_stack.size > 0):
                self.right_stack.push(self.left_stack.pop())
        return self.right_stack.pop()
            

    def peek(self) -> Any:
        if (self.right_stack.size + self.left_stack.size == 0):
            return None
        if (self.right_stack.size > 0):
            return self.right_stack.peek()
        while(self.left_stack.size > 0):
            self.right_stack.push(self.left_stack.pop())
        return self.right_stack.peek()

    @property
    def is_empty(self) -> bool:
        return (self.left_stack.size + self.right_stack.size == 0)

    @property
    def size(self):
        return self.left_stack.size + self.right_stack.size

In [5]:
# Class definition
class RobotTurtle:
    # Attributes:
    def __init__(self, name: str, speed: int=1) -> None:
        self.name: str = name
        self.speed: int = speed
        self._pos: Coordinate = Coordinate(0, 0)
        
    # property getter
    @property
    def name(self) -> str:
        return self._name
    
    # property setter
    @name.setter
    def name(self, value: str) -> None:
        if isinstance(value, str) and value != "":
            self._name: str = value
            
    # property getter
    @property
    def speed(self) -> int:
        return self._speed
    
    # property setter
    @speed.setter
    def speed(self, value: int) -> None:
        if isinstance(value, int) and value > 0:
            self._speed: int = value

    # property getter
    @property
    def pos(self) -> Coordinate:
        return self._pos
    
    # Methods:
    def move(self, direction: str) -> None:
        update: dict[str, Coordinate] = {'up' : Coordinate(self.pos.x, self.pos.y + self.speed),
                                        'down' : Coordinate(self.pos.x, self.pos.y - self.speed),
                                        'left' : Coordinate(self.pos.x - self.speed, self.pos.y),
                                        'right' : Coordinate(self.pos.x + self.speed, self.pos.y)}
        self._pos = update[direction]

        
    def tell_name(self) -> None:
        print(f"My name is {self.name}")


In [20]:
class TurtleWorld:
    valid_movements:set[str] = set('udlr')
    movement_map: dict[str, str] = {'u': 'up', 'd': 'down', 'l': 'left', 'r': 'right'}
    
    def __init__(self) -> None:
        self.turtles: dict[str, RobotTurtle] = {}
        # add a code to create a Queue for the movement
        self.movement_queue = Queue()
        
    def add_movement(self, turtle: str, movement: str) -> None:
        self.movement_queue.enqueue((turtle,movement))
    
    def run(self) -> None:
        #print(self.movement_queue.size)
        while(not self.movement_queue.is_empty):
            #print(self.movement_queue.size)
            top_el = self.movement_queue.dequeue()
            turtle, movement = top_el[0], top_el[1]
            self.move_turtle(turtle, movement)
        
    def move_turtle(self, name: str, movement: str) -> None:
        full_movement = {
            "u": "up",
            "d": "down",
            "l": "left",
            "r": "right"
        }
        for dir in movement:
            if (dir == "u" or dir == "d" or dir == "l" or dir == "r"):
                self.turtles[name].move(full_movement[dir])
    
    def add_turtle(self, name: str, speed: int) -> None:
        self.turtles[name] = RobotTurtle(name,speed)
        
    def remove_turtle(self, name: str) -> None:
        del self.turtles[name]
        
    def list_turtles(self) -> list[str]:
        return list(sorted(self.turtles.keys()))

In [21]:
world: TurtleWorld = TurtleWorld()
assert isinstance(world.movement_queue, Queue)

world.add_turtle('t1', 1)
world.add_turtle('t2', 2)
world.add_movement('t1', 'ur')
world.add_movement('t2', 'urz')
assert str(world.turtles['t1'].pos) == '(0, 0)'
assert str(world.turtles['t2'].pos) == '(0, 0)'
assert world.movement_queue.size == 2

world.run()
assert str(world.turtles['t1'].pos) == '(1, 1)'
assert str(world.turtles['t2'].pos) == '(2, 2)'

world.add_movement('t1', 'ur')
world.add_movement('t2', 'urz')

world.run()
assert str(world.turtles['t1'].pos) == '(2, 2)'
assert str(world.turtles['t2'].pos) == '(4, 4)'


###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [22]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Implement a radix sorting machine. A radix sort for base 10 integers is a *mechanical* sorting technique that utilizes a collection of bins:
- one main bin 
- 10 digit-bins

Each bin acts like a *queue* and maintains its values in the order that they arrive. The algorithm works as follows:
- it begins by placing each number in the main bin. 
- Then it considers each value digit by digit. The first value is removed from the main bin and placed in a digit-bin corresponding to the digit being considered. For example, if the ones digit is being considered, 534 will be placed into digit-bin 4 and 667 will placed into digit-bin 7. 
- Once all the values are placed into their corresponding digit-bins, the values are collected from bin 0 to bin 9 and placed back in the main bin (in that order). 
- The process continues with the tens digit, the hundreds, and so on. 
- After the last digit is processed, the main bin will contain the values in ascending order.

Create a class `RadixSort` that takes in a List of Integers during object instantiation. The class should have the following properties:
- `items`: is a List of Integers containing the numbers.

It should also have the following methods:
- `sort()`: which returns the sorted numbers from `items` as an `list` of Integers.
- `max_digit()`: which returns the maximum number of digits of all the numbers in `items`. For example, if the numbers are 101, 3, 1041, this method returns 4 as the result since the maximum digit is four from 1041. 
- `convert_to_str(items)`: which returns items as a list of Strings (instead of Integers). This function should pad the higher digits with 0 when converting an Integer to a String. For example if the maximum digit is 4, the following items are converted as follows. From `[101, 3, 1041]` to `["0101", "0003", "1041"]`.

Hint: Your implementation should make use of the generic `Queue` class, which you created, for the bins.

In [35]:
class RadixSort:
    
    def __init__(self, my_list: list[int]) -> None:
        self.items = my_list
        self.str_items = []
    
    def max_digit(self) -> int:
        ret = 0
        for i in self.items:
            max_len = len(str(i))
            ret = max(ret,max_len)
        return ret
    
    def convert_to_str(self, items: list[int]) -> list[str]:
        self.str_items = []
        mxd = self.max_digit()
        for i in items:
            edd = str(i)
            while(len(edd) < mxd):
                edd = "0" + edd
            self.str_items.append(edd)
        return self.str_items
    
    def sort(self) -> list[int]:
        self.convert_to_str(self.items)
        mxd = self.max_digit()
        main_bin = Queue()
        digit_bin = [Queue() for _ in range(10)]
        
        for item in self.str_items:
            main_bin.enqueue(item)
        
        for idx in range(mxd-1, -1, -1):
            while(main_bin.size):
                topz = main_bin.dequeue()
                digit_bin[int(topz[idx])].enqueue(topz)
            
            for i in range (0,10):
                while(digit_bin[i].size):
                    main_bin.enqueue(digit_bin[i].dequeue())
            
                    
        self.items = []
        while(main_bin.size):
            self.items.append(int(main_bin.dequeue()))
        
        #print(self.items)
        return self.items
                    
        

In [36]:
list1: RadixSort = RadixSort([101, 3, 1041])
assert list1.items == [101,3,1041]
assert list1.max_digit() == 4
assert list1.convert_to_str(list1.items) == ["0101", "0003", "1041"]
ans: list[int] = list1.sort()
print(ans)
assert ans == [3, 101, 1041]
list2: RadixSort = RadixSort([23, 1038, 8, 423, 10, 39, 3901])
assert list2.sort() == [8, 10, 23, 39, 423, 1038, 3901]
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[3, 101, 1041]


In [37]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** Write a class called `EvaluateFraction` that evaluates postfix notation implemented using Stack and Queue data structures. Postfix notation is a way of writing expressions without using parenthesis. For example, the expression `(1+2)*3` would be written as `1 2 + 3 *`. The class `EvaluateFraction` has the following method:
- `input(inp)`: which pushes the input input one at a time. For example, to create a postfix notation `1 2 + 3 *`, we can call this method repetitively, e.g. `e.input('1'); e.input('2'); e.input('+'); e.input('3'); e.input('*')`. Notice that the input is of String data type. 
- `evaluate()`: which returns the output of the expression.
- `get_fraction(inp)`: which takes in an input string and returns a `Fraction` object. 

Postfix notation is evaluated using a Stack. The input streams from `input()` are stored in a Queue. If the output of the Queue is a number, the item is pushed into the stack. If it is an operator, we will apply the operator to the two top most item n the stacks and push the result back into the stack. 

In [23]:
class Stack:
    def __init__(self) -> None:
        self.__items: list[Any] = []
        
    def push(self, item: Any):
        self.__items.append(item)

    def pop(self) -> Any:
        if (len(self.__items) == 0):
            return None
        return self.__items.pop()

    def peek(self) -> Any:
        return self.__items[-1]

    @property
    def is_empty(self) -> bool:
        return (len(self.__items) == 0)

    @property
    def size(self):
        return len(self.__items)

In [25]:
class Queue:
    def __init__(self) -> None:
        self.left_stack: Stack = Stack()
        self.right_stack: Stack = Stack()
        
    def enqueue(self, item: Any):
        self.left_stack.push(item)

    def dequeue(self) -> Any:
        if (self.right_stack.is_empty):
            while(self.left_stack.size > 0):
                self.right_stack.push(self.left_stack.pop())
        return self.right_stack.pop()
            

    def peek(self) -> Any:
        if (self.right_stack.size + self.left_stack.size == 0):
            return None
        if (self.right_stack.size > 0):
            return self.right_stack.peek()
        while(self.left_stack.size > 0):
            self.right_stack.push(self.left_stack.pop())
        return self.right_stack.peek()

    @property
    def is_empty(self) -> bool:
        return (self.left_stack.size + self.right_stack.size == 0)

    @property
    def size(self):
        return self.left_stack.size + self.right_stack.size

In [38]:
def gcd(a: int, b: int) -> int:
    if b == 0:
        return a
    else:
        return gcd(b, a % b)


class Fraction:
    def __init__(self, num: int, den: int) -> None:
        self._num = int(num)
        if (den == 0):
            self._den = 1
        else:
            self._den = int(den)
    
    @property
    def num(self) -> int:
        return self._num
    
    @num.setter
    def num(self, val: int) -> None:
        self._num = int(val)
    
    @property
    def den(self) -> int:
        return self._den
    
    @den.setter
    def den(self, val: int) -> None:
        self._den = int(val)
    
    def __str__(self) -> str:
        ret = str(self._num) + "/" + str(self._den)
        return ret
    
    def simplify(self) -> Fraction:
        ged = gcd(self._num, self._den)
        self._num /= ged
        self._den /= ged
        return Fraction(self._num, self._den)
    
    def __add__(self, other) -> Fraction:
        num1, den1, num2, den2 = self._num, self._den, other._num, other._den
        ged = gcd(den1, den2)
        lcm = (den1*den2)/ged
        dif1, dif2 = lcm/den1, lcm/den2
        
        num1 *= dif1
        num2 *= dif2
        
        return Fraction(num1+num2,lcm).simplify()
        
    
    def __eq__(self, other) -> bool:
        self.simplify()
        if (type(other) is str):
            num1 = int(other.str.split("/").str[0])
            den1 = int(other.str.split("/").str[1])
            other = Fraction(num1,den1)
        other.simplify()
        return (self._num == other._num and self._den == other._den)
    
    def __sub__(self, other) -> Fraction:
        num1, den1, num2, den2 = self._num, self._den, other._num, other._den
        ged = gcd(den1, den2)
        lcm = (den1*den2)/ged
        dif1, dif2 = lcm/den1, lcm/den2
        
        num1 *= dif1
        num2 *= dif2
        
        return Fraction(num1-num2,lcm).simplify()
    
    def __mul__(self, other) -> Fraction:
        self.simplify()
        other.simplify()
        num1, den1, num2, den2 = self._num, self._den, other._num, other._den
        
        return Fraction(num1*num2,den1*den2).simplify()
    
    def __lt__(self, other) -> bool:
        num1, den1, num2, den2 = self._num, self._den, other._num, other._den
        ged = gcd(den1, den2)
        lcm = (den1*den2)/ged
        dif1, dif2 = lcm/den1, lcm/den2
        
        num1 *= dif1
        num2 *= dif2
        
        return (num1 < num2)
    
    def __le__(self, other) -> bool:
        num1, den1, num2, den2 = self._num, self._den, other._num, other._den
        ged = gcd(den1, den2)
        lcm = (den1*den2)/ged
        dif1, dif2 = lcm/den1, lcm/den2
        
        num1 *= dif1
        num2 *= dif2
        
        return (num1 <= num2)
    
    def __gt__(self, other) -> bool:
        num1, den1, num2, den2 = self._num, self._den, other._num, other._den
        ged = gcd(den1, den2)
        lcm = (den1*den2)/ged
        dif1, dif2 = lcm/den1, lcm/den2
        
        num1 *= dif1
        num2 *= dif2
        
        return (num1 > num2)
    
    def __ge__(self, other) -> bool:
        num1, den1, num2, den2 = self._num, self._den, other._num, other._den
        ged = gcd(den1, den2)
        lcm = (den1*den2)/ged
        dif1, dif2 = lcm/den1, lcm/den2
        
        num1 *= dif1
        num2 *= dif2
        
        return (num1 >= num2)

In [52]:
class EvaluateFraction:

    operands: str = "0123456789"
    operators: str = "+-*/"
    
    def __init__(self) -> None:
        self.expression: Queue = Queue()
        self.stack: Stack = Stack()
    
    def input(self, item: str) -> None:
        self.expression.enqueue(item)
    
    def evaluate(self) -> Fraction:
        val, eval = Fraction(0,0), 0
        while (self.expression.size):
            item = self.expression.dequeue()
            if ("/" in item):
                self.stack.push(self.get_fraction(item))
            else:
                if (eval == 0):
                    val = self.get_fraction(self.stack.pop())
                    eval = 1
                
                val = self.process_operator(val,self.get_fraction(self.stack.pop()),item)
                
        return val
    
    def get_fraction(self, inp: str) -> Fraction:
        if (type(inp) is Fraction):
            return inp
        #print(inp, type(inp))
        parts = inp.split("/")
        return Fraction(int(parts[0]), int(parts[1]))
    
    def process_operator(self, op1: Fraction, op2: Fraction, op: str) -> Fraction:
        if (op == "+"):
            return op1 + op2
        elif (op == "-"):
            return op1 - op2
        elif (op == "*"):
            return op1*op2
        else:
            return op1/op2


In [53]:
pe: EvaluateFraction = EvaluateFraction()
pe.input("1/2")
pe.input("2/3")
pe.input("+")
assert pe.evaluate()==Fraction(7, 6)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
assert pe.evaluate()==Fraction(1, 1)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
pe.input("3/4")
pe.input("*")
assert pe.evaluate()==Fraction(3, 4)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


1/2 <class 'str'>
2/3 <class 'str'>
1/2 <class 'str'>
2/3 <class 'str'>
1/6 <class 'str'>
1/2 <class 'str'>
2/3 <class 'str'>
1/6 <class 'str'>
3/4 <class 'str'>


In [54]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** Modify HW2 so that it can work with MixedFraction. Write a class called `EvaluateMixedFraction` as a subclass of `EvaluateFraction`. You need to override the following methods:
- `get_fraction(inp)`: This function should be able to handle string input for MixedFraction such as `1 1/2` or `3/2`. It should return a `MixedFraction` object.
- `evaluate()`: This function should return `MixedFraction` object rather than `Fraction` object. 

In [55]:
class MixedFraction(Fraction):
    def __init__(self, top: int, bot: int, whole: int=0) -> None:
        num: Optional[int] = top + whole*bot
        super().__init__(num, bot)

    def get_three_numbers(self) -> tuple[int, int, int]:
        whole: Optional[int] = None
        top: Optional[int] = None
        bot: Optional[int] = None
        
        super().simplify()
        
        bot = super().den
        top = super().num
        if (top > bot):
            whole = top//bot
            top -= whole*bot
        
        return (top, bot, whole)

    def __str__(self) -> str:
        if (super().num < super().den):
            return str(int(self.num)) + "/" + str(int(self.den))
        whole = self.num // self.den
        return str(int(whole)) + " " + str(int(self.num - self.den*whole)) + "/" + str(int(self.den))

In [77]:
class EvaluateMixedFraction(EvaluateFraction):
    def get_fraction(self, inp: str) -> Fraction:
        if (type(inp) is Fraction):
            return inp
        
        whole = 0
        if (" " in inp):
            whole = int(inp.split(" ")[0])
            inp = inp.split(" ")[1]
        
        parts = inp.split("/")
        return Fraction(int(parts[0]) + whole*int(parts[1]), int(parts[1]))
        
    
    def evaluate(self) -> MixedFraction:
        answer, eval = Fraction(0,0), 0
        while (self.expression.size):
            item = self.expression.dequeue()
            if ("/" in item):
                self.stack.push(self.get_fraction(item))
                #print(self.stack.peek())
            else:
                #print("pp")
                if (eval == 0):
                    answer = self.get_fraction(self.stack.pop())
                    eval = 1
                
                answer = self.process_operator(answer,self.get_fraction(self.stack.pop()),item)
                
        return MixedFraction(answer.num, answer.den)

In [78]:
pe: EvaluateMixedFraction = EvaluateMixedFraction()
pe.input("3/2")
pe.input("1 2/3")
pe.input("+")
out: MixedFraction = pe.evaluate() 
assert out == MixedFraction(1, 6, 3)
assert isinstance(out, MixedFraction)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1 1/8")
pe.input("-")
assert pe.evaluate() == MixedFraction(1, 24)

pe.input("1 1/2")
pe.input("2 2/3")
pe.input("+")
pe.input("1 1/6")
pe.input("-")
pe.input("5/4")
pe.input("*")
assert pe.evaluate() == MixedFraction( 3, 4, 3)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [40]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW5.** Write a function that takes in a graph representation in a dictionary and convert it to an object-oriented representation using the `Graph` class. Refer to your cohort problem sets for the `Graph` and `Vertex` class definition. 

The function takes in a dictionary that has the following format:
```python
graph = {'vertex1': [neighbour1, neighbour2, ...],
         'vertex2': [neighbour1, ...]}
```

Notes:
- The graph is represented as and adjecancy list. 
- The key of the dictionary is the `id` of the vertex in the graph. 
- The value of the dictionary is a list of neighbours of that vertex.
- The element in the list of neighbours is the `id` of the neighbouring vertices. 
- The data type for `id` is a string. 

Example:
```python
graph = {"A": ["B", "C"], 
         "B": ["C", "D"],
         "C": ["D"],
         "D": ["C"], 
         "E": ["F"],
         "F": ["C"]}
```

In [56]:
class Vertex:
    def __init__(self, id_: str="") -> None:
        self.id_: str = id_
        self.neighbours: dict[Vertex, Number] = {}
    
    def add_neighbour(self, nbr_vertex: Vertex, weight: Number=0) -> None:
        self.neighbours[nbr_vertex] = weight
    
    def get_neighbours(self) -> list[Vertex]:
        return list(self.neighbours)
    
    def get_weight(self, neighbour: Vertex) -> Optional[Number]:
        return self.neighbours.get(neighbour, None)
        #return self.neighbours[neighbour]
    
    def __eq__(self, other) -> bool:
        return self.id_ == other.id_
    
    def __lt__(self, other) -> bool:
        return self.id_ < other.id_
    
    def __hash__(self) -> int:
        return hash(self.id_)
    
    def __str__(self) -> str:
        ret = "Vertex " + str(self.id_) + " is connected to:"
        tek = 0
        for vert in self.neighbours:
            if (tek == 0):
                tek = 1
                ret += " " + str(vert.id_)
            else:
                ret += ", " + str(vert.id_)
        #print(ret)
        return ret

In [57]:
class Graph:
    def __init__(self) -> None:
        self.vertices: dict[str, Vertex] = {}
        
    def _create_vertex(self, id_: str) -> Vertex:
        return Vertex(id_)
    
    def add_vertex(self, id_: str) -> None:
        self.vertices[id_] = self._create_vertex(id_)
    
    def get_vertex(self, id_: str) -> Optional[Vertex]:
        if (id_ in self.vertices):
            return self.vertices[id_]
        return None
    
    def add_edge(self, start_v: str, end_v: str, weight: Number=0) -> None:
        if (self.get_vertex(start_v) is None):
            self.add_vertex(start_v)
        
        if (self.get_vertex(end_v) is None):
            self.add_vertex(end_v)
        
        self.vertices[start_v].add_neighbour(self.get_vertex(end_v))
        
    def get_neighbours(self, id_: str) -> list[str]:
        if (id_ in self.vertices):
            ret = []
            for vert in self.vertices[id_].neighbours:
                ret.append(vert.id_)
            return ret
        else:
            return []
    
    def __contains__(self, val: str) -> bool:
        return val in self.vertices.keys()
    
    def __iter__(self):
        for k,v in self.vertices.items():
            yield v 
        
    @property
    def num_vertices(self):
        return int(len(self.vertices))


In [63]:
def create_graph_object(g: dict[str, list[str]]) -> Graph:
    output: Graph = Graph()
    
    for vert, neigh in g.items():
        if (output.get_vertex(vert) is None):
            output.add_vertex(vert)
        
        for item in neigh:
            output.add_edge(vert,item)
    
    return output

In [64]:
graph: dict[str, list[str]] = {"A": ["B", "C"], 
         "B": ["C", "D"],
         "C": ["D"],
         "D": ["C"], 
         "E": ["F"],
         "F": ["C"]}

output: Graph = create_graph_object(graph)
assert output.num_vertices == 6
assert sorted([x.id_ for x in output]) == ["A", "B", "C", "D", "E", "F"]
assert sorted(output.get_neighbours("A")) == ["B", "C"]
assert sorted(output.get_neighbours("C")) == ["D"]
assert sorted(output.get_neighbours("E")) == ["F"]
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [65]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
###
### AUTOGRADER TEST - DO NOT REMOVE
###
